In [1]:
from nltk.tree import *
import sys
sys.path.append("..")
from lib.tree import *
from lib.scan import *

## Chapter 2 Exercises 

#### <u>2.1</u>

```text
Consider the context-free grammar
  S -> S S + | S S * | a

a) Show how the string __a a + a *__ can be generated by this grammar.
  S -> S S * -> S S + S * -> a a + a *

b) Construct the parse tree for string.

In [2]:
def a(i): return Tree('S', [i])
s1 = Tree('S', [a(0), Tree('S', a(1)), 2])
s2 = Tree('S', [s1, Tree('S', a(3)), 4])

s2.pretty_print('a a + a *'.split())

         S         
      ___|_______   
     S       |   | 
  ___|___    |   |  
 S   S   |   S   | 
 |   |   |   |   |  
 a   a   +   a   * 



```text
c) What language is generated by this grammar? Justify your answer

  Postfix arithmetic of +, * on identifiers.
  Ex: The above tree is equivalent to _(a + a) * a_ in infix notation
```

#### <u>2.2</u>
```text
What language is generated by the following grammars? In each case justify your answer.

a) S -> 0 S 1 | 0 1
  => { 0^n 1^n | n > 0 }

b) S -> + S S | - S S | a
  => prefix arithmetic of +, - on identifiers

c) S -> S ( S ) S | ϵ
  => well parenthesized expressions

d) S -> a S b S | b S a S | ϵ
  => { (a + b)* | count(a) = count(b) }

e) S -> a | S + S | S S | S * | ( S )
  => Kleene algebra
```

#### <u>2.3</u>
```text
Which of the grammars in Exercise 2.2 are ambiguous?

Grammars (c), (d) and (e)

c) Consider the expression _()()_
  S -> S ( S ) S -> S ( ϵ ) ϵ -> ... -> ()()
  S -> S ( S ) S -> ϵ ( ϵ ) S -> ... -> ()()

d) Consider the expression _baba_
  S -> b S a S -> b {a S b S} a S -> ... -> b a b a
  S -> b S a S -> b S a {b S a S} -> ... -> b a b a

e) Consider the expression _aaa_
  S -> S S -> {S S} S -> ... -> a a a
  S -> S S -> S {S S} -> ... -> a a a
```
#### <u>2.4</u>
```text
Construct unambiguous context-free grammars for each of the following languages. In each case show that your grammar is correct.

a) Arithmetic expressions in postfix notation.

  S -> S S + | S S - | S S * | S S / | a

b) Left-associative lists of identifiers separated by commas.

  S -> S , a | a

c) Right-associative lists of identifiers separated by commas.

  S -> a , S | a

d) Arithmetic expressions of integers and identifiers with the four binary operators +, -, *, /.

  E -> E + T | E - T | T
  T -> T * F | T / F | F
  F -> ( E ) | int | id

e) Add unary plus and minus to the arithmetic
  
  E -> E + T | E - T | T
  T -> T * F | T / F | F
  F -> + E | - E | ( E ) | int | id
```

#### <u>2.5</u>
```text
a) Show that all binary strings generated by the following grammar have values divisible by 3.
  Hint. Use induction on the number of nodes in a parse tree

  num -> 11 | 1001 | num 0 | num num

  Definition num_nodes (b : num) -> nat :=
    match b with
    | 11 | 1001 => 1
    | n 0 => 2 + num_nodes n
    | n m => num_nodes n + num_nodes m
    end.

  We proceed by induction as stated, with _n_ being the number of nodes in the parse tree

  Base case: n = 1
    11_b = 3 divisible by 3
    1001_b = 9 divisible by 3

  Hypothesis: for all n m, n and m are divisible by 3

  Inductive cases:
    _n 0_b = 2 * n is divisible by 3 since
      3 | n -> 3 | 2 * n

    _n m_b = 2^(len m) * n + m is divisible by 3 since

      3 | m
      3 | n -> 3 | 2^(len m) * n -> 3 | 2^(len m) * n + m

b) Does the grammar generate all binary strings with the values divisible by 3

  //TODO I can't find a counter-example, and I can't think of a proof.
```

#### <u>2.6</u>
```text
Construct a context-free grammar for roman numerals.

  S -> A1 | A2 | A3 | A4
  A1 -> D1 A2 | D1 A3 | D1 A4 | D1
  A2 -> D2 A3 | D2 A4 | D2
  A3 -> D3 A4 | D3 
  A4 -> D4

  D4 -> I | II | III | IV | V | VI | VII | VIII | IX
  D3 -> X | XX | XXX | XL | L | LX | LXX | LXXX | XC
  D2 -> C | CC | CCC | CD | D | DC | DCC | DCCC | CM
  D1 -> M | MM | MMM
  
```

#### <u>2.7</u>
```text
Construct a syntax-directed translation scheme that translates arithmetic expressions form infix
notation into prefix notation. Give annotated parse trees for _9-5+2_ and _9-5*2_.

The test cases don't include parentheses. However, for infix, we need parentheses to represent the
complete set of arithmetic expressions. Therefore, I chose to implement them here.

- Grammar for infix -
  E -> E + T | E - T | T
  T -> T * F | T / F | F
  F -> ( E ) | 0 | 1 | ... | 9

- Syntax directed translation scheme -
  E -> {print('+')} E + T | {print('-')} E - T | T
  T -> {print('*')} T * F | {print('/')} T / F | F
  F -> (E)
    -> {print('0')} 0 
    -> {print('1')} 1
    ...
    -> {print('9')} 9

In [3]:
def E(s):
  e, s = T(s)
  while s != "" and s[0] in ['+', '-']:
    op, s = s[0], s[1:]
    t, s = T(s)
    e = Tree('E', ["{prints('"+op+"')}", e, op, t])
  return e, s

def T(s):
  t, s = F(s)
  while s != "" and s[0] in ['*', '/']:
    op, s = s[0], s[1:]
    f, s = F(s)
    t = Tree('T', ["{prints('"+op+"')}", t, op, f])
  return t, s

def F(s):
  c, s = s[0], s[1:]
  if c == '(':
    e, s = E(s)
    s = accept(s, ')')
    return Tree('F', ['(', e,')']), s
  else:
    return Tree('F', ["{prints('"+c+"')}", c]), s
  
exps = ["9-5+2", "9-5*2"]
for exp in exps:
  t, s = E(exp)
  prints(exp + " => ")
  act(t)
  print()
  in_order(t)

9-5+2 => +-952
                                                   E                                                 
       ____________________________________________|___________________________________________       
      |                                        E                             |                 |     
      |              __________________________|_____________________        |                 |      
      |             |                      F       |                 F       |                 F     
      |             |              ________|___    |         ________|___    |         ________|___   
{prints('+')} {prints('-')} {prints('9')}      9   -  {prints('5')}      5   +  {prints('2')}      2 

9-5*2 => -9*52
                                              E                                                      
       _______________________________________|__________________________                             
      |                      |       |         

#### <u>2.8</u>
```text
Construct a syntax-directed translation scheme that translates arithmetic expressions from postfix
notation into infix notation. Give annotated parse trees for the inputs _95-2*_ and _952*-_.

- Grammar for postfix -
  E -> E E + | E E - | E E * | E E / | T
  T -> 0
  T -> 1
  ...
  T -> 9

Observation: Postfix doesn't need parentheses to distinguish expressions, but in-fix does.
  Where do the parentheses need to be introduced in the translation scheme?

Answer: Since postfix productions dictate precedence, the lazy approach would be to
  parenthesize all expressions. However, it suffices to parenthesize + and - productions
  since they are the operators with a lower precedence when read in infix. Other solutions
  include parenthesizing the sub-expressions within * and / productions, but I think this
  is a poor choice since this can result in parentheses around digits, which is clearly
  redundant.

- Syntax-directed translation scheme -
  E -> {print('(')} E {print('+')} E {print(')')}  +
    -> {print('(')} E {print('+')} E {print(')')}  -
    -> E {print('*')} T *
    -> E {print('/')} T /
    -> T
  T -> 0 {print('0')}
  T -> 1 {print('1')}
  ...
  T -> 9 {print('9')}
  
```

In [4]:
def E(s):
  st = []
  while len(s) != 0:
    c, s = s[0], s[1:] 
    if c in ['+', '-']:
      o2 = st.pop()
      o1 = st.pop()
      st.append(Tree('E', ["{prints('(')}", o1, "{prints('"+c+"')}", o2, "{prints(')')}", c]))
    elif c in ['*', "/"]:
      o2 = st.pop()
      o1 = st.pop()
      st.append(Tree('E', [o1, "{prints('"+c+"')}", o2, c]))
    else:
      st.append(Tree('T', [c, "{prints('"+c+"')}"]))
  return st[0]

exps = ["95-2*", "952*-"]
for exp in exps:
  t = E(exp)
  prints(exp + " => ")
  act(t)
  print()
  in_order(t)

95-2* => (9-5)*2
                                                                E                                                                
                                           _____________________|______________________________________________________________   
                                          E                                                     |            |                 | 
       ___________________________________|____________________________________________         |            |                 |  
      |            T                      |            T                      |        |        |            T                 | 
      |         ___|________              |         ___|________              |        |        |         ___|________         |  
{prints('(')}  9      {prints('9')} {prints('-')}  5      {prints('5')} {prints(')')}  -  {prints('*')}  2      {prints('2')}  * 

952*- => (9-5*2)
                                                    

#### <u>2.9</u>
```text
Create a syntax-directed translation scheme that translates integers into roman numerals.

Similar to the grammar for roman numerals in 2.6, we need a format which encodes
the positional value of each digit.

- Grammar for integers (between 1 and 3999) -
  A1 -> D1 A2 | D1 0 A3 | D1 00 A4 | D1 000
  A2 -> D2 A3 | D2 0 A4 | D2 00
  A3 -> D3 D4 | D3 0
  A4 -> D4

  D1 -> 1 | 2 | 3
  D2 -> 1 | ... | 9
  D3 -> 1 | ... | 9
  D4 -> 1 | ... | 9

- Syntax-directed translation scheme -

  S -> A1 | A2 | A3 | A4
  A1 -> D1 A2 | D1 0 A3 | D1 00 D4 | D1 000
  A2 -> D2 A3 | D2 0 A4 | D2 00
  A3 -> D3 A4 | D3 0
  A4 -> D4

  D1 -> 1 {print('M')} | 2 {print('MM')} | 3 {print('MMM')}
  D2 -> 1 {print('C')} | 2 {print('CC')} | ... | 9 {print('CM')} 
  D3 -> 1 {print('X')} | 2 {print('XX')} | ... | 9 {print('XC')} 
  D4 -> 1 {print('I')} | 2 {print('II')} | ... | 9 {print('IX')}

```
#### <u>2.10</u>
```text
Construct a syntax-directed translation scheme that translates roman numerals into
integers.

- Grammar for Roman Numerals -

  S -> A1 | A2 | A3 | A4
  A1 -> D1 A2 | D1 A3 | D1 A4 | D1
  A2 -> D2 A3 | D2 A4 | D2
  A3 -> D3 A4 | D3 
  A4 -> D4

  D1 -> M | MM | MMM
  D2 -> C | CC | CCC | CD | D | DC | DCC | DCCC | CM
  D3 -> X | XX | XXX | XL | L | LX | LXX | LXXX | XC
  D4 -> I | II | III | IV | V | VI | VII | VIII | IX

- Syntax-directed translation scheme -

  S -> A1 | A2 | A3 | A4
  A1 -> D1 A2 | D1 {print('0')} A3 | D1 {print('00')} A4 | D1 {print('000')}
  A2 -> D2 A3 | D2 {print('0')} D4 | D2 {print('00')}
  A3 -> D3 D4 | D3 {print('0')}
  A4 -> D4

  D1 -> M {print('1')} | MM {print('2')} | MMM {print('3')} 
  D2 -> C {print('1')} | CC {print('2')} | ... | CM {print('9')}
  D3 -> X {print('1')} | XX {print('2')} | ... | XC {print('9')}
  D4 -> I {print('1')} | II {print('2')} | ... | IX {print('9')}

```
#### <u>2.11</u>
```text
a) S -> 0 S 1 | 0 1

In [5]:
def parse(s):
  s = S(s)
  if len(s) != 0: raise Exception("invalid input")

def S(s):
  if len(s) < 2: raise Exception("invalid input")
  elif len(s) == 2:
    accept(s, "01")
  elif s[0] != '0': 
    return s
  else:
    s = accept(s, '0')
    s = S(s)
    s = accept(s, '1')
  return s

parse("000111")

```text
b) S -> + S S | - S S | a
```

In [6]:
def parse(s):
  s = S(s)
  if len(s) != 0: raise Exception("invalid input")

def S(s):
  if s == "": raise Exception("invalid input")
  c, s = s[0], s[1:]
  if c == 'a': return s
  elif c in ['+', '-']:
    s = S(s)
    s = S(s)
  return s

parse("+a-aa")

```text
c) S -> S (S) S | ϵ

=> S -> X
   X -> (S) S X | ϵ

The start symbol only has a unit rule going to X, so X can be considered the start symbol instead, and we can then eliminate S by substitution.
=> X -> (X) X X | ϵ

This grammar is ambiguous, and can be reduced to S -> (S) S | ϵ, but I don't know how to prove that without using tools outside the current chapter scope.
```

In [7]:
def parse(s):
  s = S(s)
  if len(s) != 0: raise Exception("invalid input")

def S(s):
  if s == "" or s[0] != '(': return s
  s = accept(s, '(')
  s = S(s) 
  s = accept(s, ')')
  s = S(s)
  # As per the last remark, s should be empty at this point
  s = S(s) 
  return s

parse("((()()))(())")

#### <u>2.12</u>
```text
Construct a syntax-directed translator that verifies that the parentheses in an input string are properly balanced.

Assumptions:
  - The string can contain any symbol
  - The "translation" occurring here said string into a boolean value

- Grammar for well-parenthesized expressions -

  S -> A (S) A S A | A
  A -> any other non-terminal(s) except '(' and ')', including the empty string

- Syntax directed translation (attribute grammar) -

  S -> A (S1) A S2 A    { S.valid := S1.valid && S2.valid }
    -> A                { S.valid := true }

The simplest way to implement this is using a counter which keeps track of the
number of parentheses, incrementing on '(', decrementing on ')', and yielding True if the counter is 0 after all characters are scanned. This requires no recursion.

However, given that the question demands the translator be syntax-directed, we will do as such using the above scheme. We will use a synthesized attribute to represent the validity of the parentheses in each sub-expression. Rather than construct an annotated parse tree however, we will simply pass the attribute as a return value, and act as needed.

Note that if, for example, we know S1 is invalid, then we could return immediately. While this would be more computationally efficient, to stay true to the spirit of the question, we won't do that here.

The question *technically* implies that we take in *any* input string, whereas here, the grammar only accepts valid strings. The simple solution is that for any input other than a valid one, we consider it an invalid one. This does, however, complicate the grammar significantly. I opt to simply write the code from hereon, and leverage the functions I've developed to make a more sensible parser.

In [8]:
def isValid(s):
  s, valid = S(s)
  if len(s) != 0: return False
  return valid

def S(s):
  s = A(s)
  try: s = accept(s, '(')
  except: return s, True
  s, valid1 = S(s)
  try: s = accept(s, ')') 
  except: return s, False
  s = A(s)
  s, valid2 = S(s)
  s = A(s)
  return s, valid1 and valid2

def A(s):
  while s != "" and s[0] not in ['(',')']:
    s = s[1:]
  return s

print(isValid("(())()"))
print(isValid("(a(s()d(s)d)s(df)sfd)"))
print(isValid("())"))

True
True
False


#### <u>2.13</u>
```text
The following rules define the translation of an English word into pig Latin:
a) If the word begins with a nonempty string of consonants, move the initial consonant string to the back of the word and add the suffix AY; e.g., pig becomes igpay.
b) If the word begins with a vowel, add the suffix YAY; owl becomes owlyay.
c) U following a Q is a consonant.
d) Y at the beginning of a word is vowel if it is not followed by a vowel.
e) One-letter words are not changed.

Construct a syntax-directed translation scheme for pig Latin.

As always, we structure the grammar of the input language to reflect the branches needed for the translation scheme. The cases (c) and (d) complicate the grammar in such a way that Q, U and Y will be considered neither vowels nor consonants in some cases. Unless specified, q is a consonant, and y and u are a vowels.

- Grammar for English words -
  
  E -> Q | Y | ZDA' | XA' | L
  Q -> quD'| quD'VA | quVA | qWA | qD' | qD'VA 
  Y -> yVA | yCA
  V -> a | e | i | o | u | y
  W -> a | e | i | o | y
  X -> a | e | i | o | u
  D -> CD | ϵ
  D'-> CD
  C -> [any consonant, including q]
  Z -> [any consonant, except q]
  A -> LA | ϵ
  A'-> LA
  L -> [any letter]

Description
- A represents 0 or more letters, A' represents 1 or more letters
- D represents 0 or more consonants, D' represents 1 or more consonants
- E's productions comprise all words, broken up into the following cases, in order:
  - 2+ letter words starting with q
  - 2+ letter words starting with y
  - 2+ letter words starting with any consonant except q
  - 2+ letter words starting with any vowel except y
  - 1 letter words
- Q considers the cases:
  - 3+ letter word starting with qu followed by 1 or more consonants, before ending
  - 3+ letter word starting with qu followed by 1 or more consonants, then a vowel and potentially more letters
  - 3+ letter word starting with qu followed by a vowel
  - 2+ letter word starting with q followed by any vowel except u
  - 2+ letter word starting with q followed by 1 or more consonants, before ending
  - 2+ letter word starting with q followed by 1 or more consonants, then a vowel and potentially more letters
- Y considers the cases:
  - 2+ letter word starting with y followed by any vowel
  - 2+ letter word starting with y followed by any consonant

The reason Q requires so many cases is so that it can both capture the consonant chain and avoid ambiguity.

For conciseness, we use the variable v, c, and l to represent the matched non-terminal.

- Syntax-directed translation scheme -
  S -> E      {print(E.s)}
  E -> Q      {E.s := Q.s}    
    -> Y      {E.s := Y.s}
    -> ZDA'   {E.s := A'.s' + Z.s + D.s + 'ay'}
    -> XA'    {E.s := X.s + A'.s + 'yay'}
    -> L      {E.s := L.s}
  Q -> quD'   {Q.s := 'qu' + D'.s + 'ay'}
    -> quD'VA {Q.s := V.s + A.s + 'qu' + D'.s + 'ay'}
    -> quVA   {Q.s := V.s + A.s + 'quay'}
    -> qWA    {Q.s := W.s + A.s + 'qay'}
    -> qD'    {Q.s := 'q' + D'.cons + 'ay'}
    -> qD'VA  {Q.s := V.a + A.s + 'q' + D'.cons + 'ay'}
  Y -> yVA    {Y.s := V.a + A.s + 'yay'}
    -> yCA    {Y.s := 'y' + C.s + A.s + 'yay'}

  V -> a | e | i | o | u | y  => v {V.s := v}
  W -> a | e | i | o | y      => v {W.s := v}
  X -> a | e | i | o | u      => v {X.s := v}

  D -> CD   {D.s := C.s + D.s}
    -> ϵ    {D.s := ''}
  D'-> CD   {D.s := C.s + D.s}

  C -> [any consonant, including q] => c {C.s := c}
  Z -> [any consonant, except q]    => c {C.s := c}

  A -> LA   {A.s := L.s + A.s}
    -> ϵ    {A.s := ''}
  A'-> LA   {A.s := L.s + A.s}
  L -> [any letter] => l {L.s := l}

As a program, this translation scheme can be made much simpler by using lookaheads and if-else statements. Not to mention, the non-exceptional cases for Q and Y effectively behave the same as the other cases of the consonant and vowel families respectively. I think this is worth demonstrating.

In [9]:
# We dont need the exceptional sets W, X, and Z because we can just use if-elses to
# catch the exceptions first
vowels = ['a', 'e', 'i', 'o', 'u', 'y']
cons = ['q','w','r','t','p','s','d','f','g','h','j','k','l','z','x','c','v','b','n','m']

def parse(s):
  for s in str.split(s):
    print(E(s)+' ', end='')
  print()

def E(s):
  if s == "": raise Exception("empty string")
  if len(s) == 1: return s
  
  # At this point, s has at least length two
  la = s[0]
  if la == 'q': return Q(s)
  elif la == 'y': return Y(s)
  elif la in vowels: return s + 'yay'
  elif la in cons:
    c, s = D(s)
    return s + c + 'ay'
  
def Q(s):
  if s[:2] == 'qu':
    c, s = D(s[2:])
    return s + 'qu' + c + 'ay'
  else:
    c, s = D(s)
    return s + c + 'ay'
    
def Y(s):
  if s[1] in vowels:
    return s[1:] + 'yay'
  else:
    return s + 'yay'

def D(s):
  c = ''
  while s[0] in cons:
    c, s = c + s[0], s[1:]
  return c, s
    
parse('the quick brown fox jumps over the lazy dog')
parse('a year of yttrium decay')
parse('the qi is off')

ethay ickquay ownbray oxfay umpsjay overyay ethay azylay ogday 
a earyay ofyay yttriumyay ecayday 
ethay iqay isyay offyay 


```text
We can see the similarities in the non-exceptional cases for Q and Y versus the base cases. These could acknowledged in the grammar, but again, this would make the grammar much more complicated than it need be. The limitation with grammars is that cases must be disjoint, lest they result in an ambiguous grammar where two rules with different semantic actions can be triggered by the same input string.

Here is the code, further simplified.

In [10]:
# We dont need the exceptional sets W, X, and Z because we can just use if-elses to catch the exceptions first
vowels = ['a', 'e', 'i', 'o', 'u', 'y']
cons = ['q','w','r','t','p','s','d','f','g','h','j','k','l','z','x','c','v','b','n','m']

def parse(s):
  for s in str.split(s):
    print(E(s)+' ', end='')
  print()

def E(s):
  if s == "": raise Exception("empty string")
  if len(s) == 1: return s
  
  # At this point, s has at least length two
  if s[0:2] == 'qu':
    c, s = D(s[2:])
    return s + 'qu' + c + 'ay'
  
  la = s[0]
  if la == 'y' and s[1] in vowels: 
    return s[1:] + 'yay'
  if la in vowels:
    return s + 'yay'
  elif la in cons:
    c, s = D(s)
    return s + c + 'ay'
  
def D(s):
  c = ''
  while s[0] in cons:
    c, s = c + s[0], s[1:]
  return c, s
    
parse('the quick brown fox jumps over the lazy dog')
parse('a year of yttrium decay')
parse('the qi is off')

ethay ickquay ownbray oxfay umpsjay overyay ethay azylay ogday 
a earyay ofyay yttriumyay ecayday 
ethay iqay isyay offyay 


#### <u>2.14</u>
```text
In the programming language C, the for-statement has the form:

  for (expr1; expr2; expr3) stmt1

The first expression is executed before the loop; it is typically used for initializing the loop index. The second expression is a test made before each iteration of the loop; the loop is exited if the expression becomes 0. The loop itself consists of the statement { stmt expr3; }. The third expression is executed at the end of each iteration; it is typically used to increment the loop index. The meaning of the for-statement is similar to

  expr1; while (expr2) { stmt1 expr3; }

Construct a syntax-directed translation scheme to translate C for-statements into stack-machine code.

Based on the stack-machine code for a while-loop in the textbook, the goal would be to create the following translation:

{ out := newlabel;
  loop := newlabel;
  stmt.t := expr1.t || label loop || expr2.t || gofalse out || stmt1.t || expr3.t || goto loop || label out }

However an issue with this is that stmt1 and expr3 are emitted out of order. We need a way to have the resulting translation execute these emissions in reverse order, and we can do this using extra labels, like so:

  code for expr1
  label loop
  code for expr2
  gofalse out
  
  goto fst
  label snd
  code for expr3
  goto loop

  label fst
  code for stmt1
  goto snd

  label out

Using all-caps for non-terminals:

STMT -> 
  for ( 
  EXPR      { loop := newlabel; emit('label', loop) }
  ; EXPR    { out := newlabel; emit('gofalse', out) }
  ;         { fst := newlabel; snd := newlabel; emit('goto', fst); emit('label', snd) } 
  EXPR      { emit('goto', loop); emit('label', fst) }
  ) STMT_1  { emit('goto', snd); emit('label', out) }
  

#### <u>2.15</u>
```text
Consider the following for-statement:
  for i := | step 10-j until 10*j do j := j + 1

Three semantic definitions can be given for this statement. One possible meaning is that the limit 10*j and increment 10-j are to be evaluated once before the loop, as in PL/I. For example, if j = 5 before the loop, we would run through the loop ten times and exit. A second, completely different meaning would ensue if we are required to evaluate the limit and increment every time through the loop. For example, if j = 5 before the loop, the loop would never terminate. A third meaning is given by languages such as Algol. When the increment is negative, the test made for termination of the loop is i < 10*j, rather than i > 10*j. For each of these three semantic definitions construct a syntax-directed translation scheme to translate these for-loops into stack-machine code.

Note: In this question, I assume that the starting value given to i by this construct is 0, and thus that the limit is exclusive, such that the first case results in the number of iterations described. I also assume that the variable i can be accessed and manipulated within the body of the loop, at that the increment is applied to whatever the current value of i is.

The statement consists of two expressions and a statement:
  for i := | step expr1 until expr2 do stmt

There is also the variable i in the local context.

-- Semantic 1: Evaluation of expressions before iteration --

In this first version, the implementation requires that we use the same limit and increment on each run of the loop. Thus we would need to maintain copies of both values using only stack operations. Doing this while still being able to access both values on the stack is impossible without losing one or the other. The only options then, are to maintain the context which created these values, i.e. the value of j before the loop, or to generate all the values of i ahead of time before running the body.The first is only possible because j is the only variable that we'd need to maintain a history for.


The best option then, is to create a block of code that will generate all the values of i ahead of time before running the body.

The process is as follows:

generate a loop

push 

push j onto the stack
update i = requires step value -> requires j's original value
check i vs limit = requires limit value -> requires j's original value
execute body = requires i and j current values





#### <u>2.16</u>
Consider the following grammar fragment for if-then- and if-then-else-statements:

stmt -> if expr then stmt
  | if expr then stmt else stmt
  | other

where other stands for the other statements in the language.

a) Show that this grammar is ambiguous.
b) Construct an equivalent unambiguous grammar that associates each else with the closest previous unmatched then.
c) Construct a syntax-directed translation scheme based on this grammar to translate conditional statements into stack machine code.